<a href="https://colab.research.google.com/github/BrownParticleAstro/dmtools/blob/podman-python-dash/notebooks/DMTOOLSGraphDemo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 24.0 MB/s eta 0:00:00


In [2]:
!pip install --upgrade certifi

In [3]:
import ssl
import urllib
context = ssl._create_unverified_context()
from urllib.request import Request, urlopen

In [4]:
import dash
from dash import Dash
import json
import requests
import pathlib
import sqlalchemy
from sqlalchemy import text
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from dash import Dash, dash_table
from dash import dcc, callback, html
from dash import Input, Output, State

from itertools import cycle

from decimal import *

In [5]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Decimal was to maintain accuracy of cross section data

In [6]:
def setup_decimal_precision():
    getcontext().prec = 24
    print(Decimal(748327402479023).sqrt())
setup_decimal_precision()

27355573.5176402945763287


In [7]:
# Build App
# app = Dash(__name__)

In [8]:
# colors
# palette = cycle(px.colors.qualitative.Bold)

# Set Up Request Header

In [9]:
#request_header = {'dmtool-userid': '1' ,'dmtool-apikey': 'CC0EF1DD_C9AE_11EE_BCAD_0EC372396BDC'} # dev server
#request_header = {'dmtool-userid': '1' ,'dmtool-apikey': '81D34E22_CC0F_11EE_8826_FE852FEDCE00'} ## prod server
# 58E88A3E_37DC_11EF_BCE0_FADBD26D994C

def create_request_header(dmtool_userid_in,dmtool_apikey_in ):
    request_header_out = {'dmtool-userid': dmtool_userid_in ,'dmtool-apikey': dmtool_apikey_in}
    return request_header_out
request_header = create_request_header('1','58E88A3E_37DC_11EF_BCE0_FADBD26D994C' )

# Data Introduction

    data is the Raw Upload Format
    data_data is the parsed data
    data_about is the summary fields only
    data_display is a record for each data series in the dataset and how it appears in a chart


The api have been designed to retrieve data based on:
    plot
    data
    id

The routes have been named as follows

    action + how/what + subject

This makes understanding code easier and allowed for dynamic creation.

Plural subjects for read_all seemed unecessary.

> Add blockquote

> Add blockquote





# Set Up Request URL

In [10]:
api_server = "https://dmtools.het.brown.edu/"
data_api = "dmtool/fastapi_data/open/data/"

fastapi_url_data_data = api_server + data_api + "read_all_data_data/"

fastapi_url_read_plot_data_data = api_server + data_api + "read_plot_data_data/"

fastapi_url_data_about = api_server + data_api + "read_all_data_about"
fastapi_url_read_all_data_display = api_server + data_api +  "read_all_data_display/"

fastapi_url_read_plot_data_display = api_server + data_api +  "read_plot_data_display/"


fastapi_url_plots = api_server + data_api + "read_all_plot/"
#/dmtool/fastapi_data/open/data/plots

In [11]:
fastapi_url_data_data

'https://dmtools.het.brown.edu/dmtool/fastapi_data/open/data/read_all_data_data/'

In [12]:
#fastapi_url_data_data = "https://dev1.dmtool.info/dmtool/fastapi_data/open/data/data_datas/"
#fastapi_url_data_about = "https://dev1.dmtool.info/dmtool/fastapi_data/open/data/data_abouts"
#fastapi_url_data_display = "https://dev1.dmtool.info/dmtool/fastapi_data/open/data/data_displays/"
#fastapi_url_plots = "https://dev1.dmtool.info/dmtool/fastapi_data/open/data/plots/"

In [13]:
#all_subject_api = '/dmtool/fastapi_data/internal/data/' + self.page_subject + 's'
#api_server = "http://container_fastapi_data_1:8014
#all_subject_request_api = api_server + all_subject_api
#print("PD : all_subject_api >>>>>>>>>>>>", all_subject_request_api)
#print("PD : all_subject user_header >>>>>>>>>>>>", self.dmtool_user_header)

In [14]:
print(fastapi_url_plots)

https://dmtools.het.brown.edu/dmtool/fastapi_data/open/data/read_all_plot/


In [15]:
## List All Plots Available

## List All Plots Available

In [16]:
def get_all_owned_and_open_plots(fastapi_url_plots_in, request_header_in):
    #r = requests.get(fastapi_url_plots, headers = request_header)
    request = Request(fastapi_url_plots_in, headers=request_header)
    r = urllib.request.urlopen(request, context=context)

    #r = urllib.request.urlopen(fastapi_url_plots,context=context, headers = request_header)
    #print(r.read())
    string = r.read().decode('utf-8')
    all_plots_json_obj = json.loads(string)
    #all_plots_json_obj[1]
    all_plots_df_ret = pd.DataFrame(all_plots_json_obj)
    all_plots_df_ret['row_id'] = all_plots_df_ret.index
    #all_plots_df.head(10)
    return all_plots_df_ret
all_plots_df = get_all_owned_and_open_plots(fastapi_url_plots, request_header)
all_plots_df.columns

Index(['x_max', 'updated_at', 'y_min', 'archived_at', 'subject', 'y_max',
       'plot_png', 'id', 'x_units', 'legend_png', 'old_plot_id', 'y_units',
       'plot_eps', 'open', 'user_id', 'legend_eps', 'name', 'old_user_id',
       'no_id', 'x_min', 'created_at', 'row_id'],
      dtype='object')

In [17]:
all_plots_df[['id', 'old_plot_id', 'y_units','open', 'user_id','name', 'created_at', 'row_id']]

,id,old_plot_id,y_units,open,user_id,name,created_at,row_id
0,20,53,cm2,1,None,XENON10_for_thesis,2009-10-14T07:48:53,0
1,26,66,cm2,1,None,XENON10_LUX_compare,2009-11-19T14:22:11,1
2,33,73,cm2,1,None,XENON10_LUX_ZEPLINIII,2009-11-23T21:12:03,2
3,54,107,cm2,1,None,CDMS_new_results,2010-01-04T18:59:18,3
4,74,148,cm2,1,None,CoGeNT_plus_Xenon_100_New,2010-02-05T00:29:07,4
...,...,...,...,...,...,...,...,...
132,2902,4374,cm2,1,None,"SuperK, IceCube, and Xenon10",2019-06-28T18:23:11,132
133,2920,4396,cm2,1,None,LUX Run3+4 Combined SDn 90% Upper Limit,2019-08-27T07:10:37,133
134,3009,4523,cm2,1,None,Fermi and Xenon and SUSY,2020-06-03T15:38:06,134
135,3094,4643,cm2,1,None,XENON1T_DDnSI,2021-06-15T08:02:13,135


In [18]:
def get_new_plot_id_from_old_id(all_plots_df_in, old_plot_id_in):
    old_plot_details_ret = all_plots_df[['name','x_units', 'id', 'y_units','old_plot_id',  'x_min', 'x_max', 'y_min', 'y_max']] \
    [all_plots_df['old_plot_id']==old_plot_id_in]
    return old_plot_details_ret
old_plot_details = get_new_plot_id_from_old_id(all_plots_df,73)
old_plot_details

,name,x_units,id,y_units,old_plot_id,x_min,x_max,y_min,y_max
2,XENON10_LUX_ZEPLINIII,GeV/c^2,33,cm2,73,1,10000,-48,-32


## Enter Plot ID Here

In [19]:
plot_id = 33
this_plot = all_plots_df[all_plots_df['id']==plot_id]
this_plot

,x_max,updated_at,y_min,archived_at,subject,y_max,plot_png,id,x_units,legend_png,...,plot_eps,open,user_id,legend_eps,name,old_user_id,no_id,x_min,created_at,row_id
2,10000,2010-01-04T19:03:59,-48,1980-01-01T00:00:00,plot,-32,None,33,GeV/c^2,None,...,None,1,None,None,XENON10_LUX_ZEPLINIII,None,0,1,2009-11-23T21:12:03,2


# Extract all Data About Plot

In [20]:
def get_data_about_plot(all_plots_df_in,plot_id_in):
    this_plot = all_plots_df_in[all_plots_df_in['id']==plot_id_in]
    #this_plot
    start_x_range_ret = int(this_plot.iloc[0]['x_min'])
    stop_x_range_ret = int(this_plot.iloc[0]['x_max'])
    start_y_range_ret = int(this_plot.iloc[0]['y_min'])
    stop_y_range_ret = int(this_plot.iloc[0]['y_max'])
    all_plots_df_in[all_plots_df_in['id'] == plot_id_in][['id','old_plot_id','name']]
    plot_name_df = this_plot[['name','old_plot_id']]
    plot_name_ret = plot_name_df.iloc[0]['name']
    old_plot_id_ret = plot_name_df.iloc[0]['old_plot_id']
    return start_x_range_ret,stop_x_range_ret, start_y_range_ret, stop_y_range_ret, plot_name_ret, old_plot_id_ret

start_x_range,stop_x_range, start_y_range, stop_y_range, plot_name, old_plot_id = get_data_about_plot(all_plots_df, plot_id)

#get_data_about_plot(all_plots_df, 828)
    #return start_x_range_ret
#start_x_range, stop_x_range,start_y_range, stop_y_range,all_plots_df_in[all_plots_df_in['id'] == plot_id_in][['id','old_plot_id','name']]
#    plot_name_df = this_plot[['name','old_plot_id']]
#    plot_name_ret = plot_name_df.iloc[0]['name']
#    old_plot_id_ret = plot_name_df.iloc[0]['old_plot_id']#
#x

# Get All Data About Records (Limit Summaries)

In [21]:
#def get_all_data_abouts(fastapi_url_data_about_in, request_header_in):
#    request = Request(fastapi_url_data_about_in, headers=request_header_in)
#    r = urllib.request.urlopen(request, context=context)
    #r = urllib.request.urlopen(fastapi_url_plots,context=context, headers = request_header)
    #print(r.read())
#    string = r.read().decode('utf-8')
#    data_about_json_obj = json.loads(string)
#    #data_about_json_obj[1]
#    data_about_df_ret = pd.DataFrame(data_about_json_obj)
#    data_about_df_ret['row_id'] = data_about_df_ret.index
#    data_about_df_ret[['data_id','data_label']].head(5)
#    return data_about_df_ret
#data_about_df = get_all_data_abouts(fastapi_url_data_about, request_header)
#data_about_df.head(6)

## Get Display Data for Plot (Limit Displays)

In [22]:
#fastapi_url_data_display

In [23]:
def get_data_displays_for_plot(fastapi_url_data_display_in, plot_id_in, request_header_in):
    request = Request(fastapi_url_read_plot_data_display + '?id_in=' + str(plot_id_in), headers=request_header_in)
    r = urllib.request.urlopen(request, context=context)

    #r = urllib.request.urlopen(fastapi_url_plots,context=context, headers = request_header)
    #print(r.read())
    string = r.read().decode('utf-8')
    data_display_json_obj = json.loads(string)
    data_display_df_ret = pd.DataFrame(data_display_json_obj)
    data_display_df_ret['row_id'] = data_display_df_ret.index
    data_display_df_ret[['plot_id', 'data_id', 'trace_name']].head(5)
    return data_display_df_ret
data_display_df = get_data_displays_for_plot(fastapi_url_read_plot_data_display, plot_id, request_header)
data_display_df.head(5)

,id,subject,name,limit_id,data_id,old_plot_id,plot_id,series_id,data_label,x_units,...,result_type,mode_value,line_width,line_format,fill_style,opacity_style,marker_size,marker_color,updated_at,row_id
0,3495,data_display,None,273,255,73,33,None,"XENON10, 2007, first results, 58.6 live days, SI",None,...,None,None,None,solid,none,None,None,blue,2024-06-20T13:22:26,0
1,3496,data_display,None,136,134,73,33,None,LUX 300 kg LXe Projection (Jul 2007) - Offline,None,...,None,None,None,dash,none,None,None,cyan,2024-06-20T13:22:26,1
2,3497,data_display,None,136,134,73,33,None,LUX 300 kg LXe Projection (Jul 2007) - Offline,None,...,None,None,None,dash,none,None,None,cyan,2024-06-20T13:22:26,2
3,3498,data_display,None,300,267,73,33,None,"ZEPLIN III, 2009, first science run, 847kg-day...",None,...,None,None,None,solid,none,None,None,green,2024-06-20T13:22:26,3


In [24]:
#data_display_df = data_display_df[data_display_df.data_id==5]

## Get Data for the Plot (Detailed Data Parsed into data_data table)

In [25]:
fastapi_url_read_plot_data_data + '?id_in=' + str(plot_id)

'https://dmtools.het.brown.edu/dmtool/fastapi_data/open/data/read_plot_data_data/?id_in=33'

In [26]:
def convert_string_to_decimal(string_in):
  try:
     decimal_out = Decimal(string_in)
  except:
     if string_in == None:
        decimal_out = Decimal(0)
     else:
        decimal_out = Decimal(0)
  return decimal_out

In [27]:
fastapi_url_read_plot_data_data

'https://dmtools.het.brown.edu/dmtool/fastapi_data/open/data/read_plot_data_data/'

In [28]:
def get_data_data_for_plot(fastapi_url_data_data_in, plot_id_in, request_header_in):
    request = Request(fastapi_url_data_data_in + '?id_in=' + str(plot_id_in), headers=request_header_in)
    r = urllib.request.urlopen(request, context=context)

    #r = urllib.request.urlopen(fastapi_url_plots,context=context, headers = request_header)
    #print(r.read())
    string = r.read().decode('utf-8')
    data_data_json_obj = json.loads(string)
    #json_obj['data']
    #response_data_data_df_ret = pd.DataFrame(data_data_json_obj['data'])
    #response_data_data_df_ret['row_id'] = response_data_data_df_ret.index
    #response_data_data_df_ret.head(5)
    # the dtype string was to ensure all the significant figures are brought into the workbook
    # the x y data is also stored in the database as columns, but we could not ensure the data was extracted and sent fully by fastapi
    data_data_df_ret = pd.DataFrame(data_data_json_obj, columns=['data_id','trace_id','series_id','raw_x', 'raw_y', 'x_rescale', 'y_rescale'], dtype='str')
    #data_data_df.dtypes
    data_data_df_ret[['raw_y_n', 'raw_y_e']] = data_data_df_ret['raw_y'].str.split('e', n=1, expand=True)
    #data_data_df['cross_sections_n'] = data_data_df['raw_y_n'].apply(Decimal)
    # convert text to scientific values
    data_data_df_ret['cross_sections_n'] = data_data_df_ret['raw_y_n'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret))
    data_data_df_ret['cross_sections_e'] = data_data_df_ret['raw_y_e'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret))
    # rescale numbers if required
    data_data_df_ret['x_rescale_value'] = data_data_df_ret['x_rescale'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret)) ## cross section
    data_data_df_ret['y_rescale_value'] = data_data_df_ret['y_rescale'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret)) ## mass
    # calculates cross sections and masses
    data_data_df_ret['cross_sections'] = data_data_df_ret['cross_sections_n'] * (10 ** data_data_df_ret['cross_sections_e']) * (10 ** data_data_df_ret['y_rescale_value'])
    data_data_df_ret['masses'] = data_data_df_ret['raw_x'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret)) * (10** data_data_df_ret['x_rescale_value'])
    # convert other columns to int
    data_data_df_ret['data_id'] = data_data_df_ret['data_id'].astype(int)
    data_data_df_ret['trace_id'] = data_data_df_ret['trace_id'].astype(int)
    data_data_df_ret['series_id'] = data_data_df_ret['series_id'].astype(int)

    #data_data_df_ret[['data_id','trace_id','series_id','masses','cross_sections']].head(10)
    return data_data_df_ret

data_data_df = get_data_data_for_plot(fastapi_url_read_plot_data_data, plot_id, request_header)
data_data_df.head(5)

,data_id,trace_id,series_id,raw_x,raw_y,x_rescale,y_rescale,raw_y_n,raw_y_e,cross_sections_n,cross_sections_e,x_rescale_value,y_rescale_value,cross_sections,masses
0,134,1,0,10.593,5.9693e-44,1,1,5.9693,-44,5.9693,-44,1,1,5.96930E-43,105.930
1,134,1,1,11.22,2.6973e-44,1,1,2.6973,-44,2.6973,-44,1,1,2.69730E-43,112.20
2,134,1,2,11.885,1.4709e-44,1,1,1.4709,-44,1.4709,-44,1,1,1.47090E-43,118.850
3,134,1,3,12.589,8.9193e-45,1,1,8.9193,-45,8.9193,-45,1,1,8.91930E-44,125.890
4,134,1,4,13.335,5.8066e-45,1,1,5.8066,-45,5.8066,-45,1,1,5.80660E-44,133.350


## Convert Text Scientific Numbers into Actual Scientific Numbers

## Looking for Min and Max Values for Axes (Requires help)

In [29]:
from math import log10, floor
def find_exp(number_in):
  try:
    base10 = log10(abs(number_in))
  except:
    base10 = log10(1)

  return floor(base10)


#data_data_df.head(5)
min_cross_sections = data_data_df['cross_sections'].min()
max_cross_sections = data_data_df['cross_sections'].max()
min_masses = data_data_df['masses'].min()
max_masses = data_data_df['masses'].max()

print('min y :' ,min_cross_sections)
print('max y :' ,max_cross_sections)
print('min x :' ,min_masses)
print('max x :' ,max_masses)

#start_x_range = find_exp(min_masses)
#stop_x_range = find_exp(max_masses)
#start_y_range = find_exp(min_cross_sections)
#stop_y_range = find_exp(max_cross_sections)

print(start_x_range)
print(stop_x_range)
print(start_y_range)
print(stop_y_range)

min y : 4.30440E-45
max y : 1.9720E-39
min x : 70
max x : 100000
1
10000
-48
-32


## Creating Blank Chart (This will be used in the app)

In [30]:
def create_blank_chart_for_plot(plot_name_in, plot_id_in, old_plot_id_in):
    y_title_text = r"$\text{WIMP Mass [GeV}/c^{2}]$"
    x_title_text = r"$\text{Cross Section [cm}^{2}\text{] (normalized to nucleon)}$"
    plot_title = plot_name_in + " - Plot Reference:" + str(plot_id_in) + " - Old Plot ID:" + str(old_plot_id_in)
    #plot_title
    ## create empty chart
    fig_chart_out = go.Figure(data=[go.Scatter(x=[], y=[])])
    fig_chart_out.update_layout( autosize=False, width=800, height=800, )
    fig_chart_out.update_layout(xaxis_range=[-1,-4])
    fig_chart_out.update_layout(yaxis_range=[-1,-4])
    fig_chart_out.update_layout(
            margin=dict(l=0, r=0, t=20, b=0),
            paper_bgcolor="LightSteelBlue",)

    fig_chart_out.update_layout(
        title=dict(text=plot_title , font=dict(size=18), automargin=True, yref='paper')
    )
    fig_chart_out.update_xaxes(
        title_text=x_title_text,
        #type="log"
        type="linear"
    )
    fig_chart_out.update_yaxes(
        title_text=y_title_text,
        type="log"
        #type="linear"
    )
    return fig_chart_out

fig_chart = create_blank_chart_for_plot(plot_name,plot_id, old_plot_id)
fig_chart.show()


In [31]:
'''
            the followig maybe used when creating new plots
            ===============================================
            spin_dependency = row_data['spin_dependency']
            result_type = row_data['result_type']
            marker_size = 4
            marker_color = row['trace_color_default']
            if spin_dependency == 'SI':
                trace_mode_value = 'lines'
                line_color = row['trace_color_default']
                line_style = dict(color=line_color)
            elif spin_dependency == 'SD':
                trace_mode_value = 'lines'
                line_width = 4
                line_format = 'dash'
                line_color = row['trace_color_default']
                line_style = dict(width= line_width ,dash= line_format ,color = line_color )
            else:
                mode_value = 'lines'
                line_style = None

            if result_type == 'Th':
                fill_style = 'toself'
                fill_color = row['trace_color_default']
                opacity_style = 0.4
            else:
                fill_style = None
                fill_color = None
                opacity_style = 1
            trace_color_default = row_data['trace_color_default']


    @app.callback([Output('graph2', 'figure')],
            [Input('graph', 'relayoutData')], # this triggers the event
            [State('graph2', 'figure')])
    def zoom_event(relayout_data, *figures):
        outputs = []
        for fig in figures:
            try:
                fig['layout']["xaxis"]["range"] = [relayout_data['xaxis.range[0]'], relayout_data['xaxis.range[1]']]
                fig['layout']["xaxis"]["autorange"] = False
            except (KeyError, TypeError):
                fig['layout']["xaxis"]["autorange"] = True

            outputs.append(fig)

        return outputs

 The 'symbol' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            [0, '0', 'circle', 100, '100', 'circle-open', 200, '200',
            'circle-dot', 300, '300', 'circle-open-dot', 1, '1',
            'square', 101, '101', 'square-open', 201, '201',
            'square-dot', 301, '301', 'square-open-dot', 2, '2',
            'diamond', 102, '102', 'diamond-open', 202, '202',
            'diamond-dot', 302, '302', 'diamond-open-dot', 3, '3',
            'cross', 103, '103', 'cross-open', 203, '203',
            'cross-dot', 303, '303', 'cross-open-dot', 4, '4', 'x',
            104, '104', 'x-open', 204, '204', 'x-dot', 304, '304',
            'x-open-dot', 5, '5', 'triangle-up', 105, '105',
            'triangle-up-open', 205, '205', 'triangle-up-dot', 305,
            '305', 'triangle-up-open-dot', 6, '6', 'triangle-down',
            106, '106', 'triangle-down-open', 206, '206',
            'triangle-down-dot', 306, '306', 'triangle-down-open-dot',
            7, '7', 'triangle-left', 107, '107', 'triangle-left-open',
            207, '207', 'triangle-left-dot', 307, '307',
            'triangle-left-open-dot', 8, '8', 'triangle-right', 108,
            '108', 'triangle-right-open', 208, '208',
            'triangle-right-dot', 308, '308',
            'triangle-right-open-dot', 9, '9', 'triangle-ne', 109,
            '109', 'triangle-ne-open', 209, '209', 'triangle-ne-dot',
            309, '309', 'triangle-ne-open-dot', 10, '10',
            'triangle-se', 110, '110', 'triangle-se-open', 210, '210',
            'triangle-se-dot', 310, '310', 'triangle-se-open-dot', 11,
            '11', 'triangle-sw', 111, '111', 'triangle-sw-open', 211,
            '211', 'triangle-sw-dot', 311, '311',
            'triangle-sw-open-dot', 12, '12', 'triangle-nw', 112,
            '112', 'triangle-nw-open', 212, '212', 'triangle-nw-dot',
            312, '312', 'triangle-nw-open-dot', 13, '13', 'pentagon',
            113, '113', 'pentagon-open', 213, '213', 'pentagon-dot',
            313, '313', 'pentagon-open-dot', 14, '14', 'hexagon', 114,
            '114', 'hexagon-open', 214, '214', 'hexagon-dot', 314,
            '314', 'hexagon-open-dot', 15, '15', 'hexagon2', 115,
            '115', 'hexagon2-open', 215, '215', 'hexagon2-dot', 315,
            '315', 'hexagon2-open-dot', 16, '16', 'octagon', 116,
            '116', 'octagon-open', 216, '216', 'octagon-dot', 316,
            '316', 'octagon-open-dot', 17, '17', 'star', 117, '117',
            'star-open', 217, '217', 'star-dot', 317, '317',
            'star-open-dot', 18, '18', 'hexagram', 118, '118',
            'hexagram-open', 218, '218', 'hexagram-dot', 318, '318',
            'hexagram-open-dot', 19, '19', 'star-triangle-up', 119,
            '119', 'star-triangle-up-open', 219, '219',
            'star-triangle-up-dot', 319, '319',
            'star-triangle-up-open-dot', 20, '20',
            'star-triangle-down', 120, '120',
            'star-triangle-down-open', 220, '220',
            'star-triangle-down-dot', 320, '320',
            'star-triangle-down-open-dot', 21, '21', 'star-square',
            121, '121', 'star-square-open', 221, '221',
            'star-square-dot', 321, '321', 'star-square-open-dot', 22,
            '22', 'star-diamond', 122, '122', 'star-diamond-open',
            222, '222', 'star-diamond-dot', 322, '322',
            'star-diamond-open-dot', 23, '23', 'diamond-tall', 123,
            '123', 'diamond-tall-open', 223, '223',
            'diamond-tall-dot', 323, '323', 'diamond-tall-open-dot',
            24, '24', 'diamond-wide', 124, '124', 'diamond-wide-open',
            224, '224', 'diamond-wide-dot', 324, '324',
            'diamond-wide-open-dot', 25, '25', 'hourglass', 125,
            '125', 'hourglass-open', 26, '26', 'bowtie', 126, '126',
            'bowtie-open', 27, '27', 'circle-cross', 127, '127',
            'circle-cross-open', 28, '28', 'circle-x', 128, '128',
            'circle-x-open', 29, '29', 'square-cross', 129, '129',
            'square-cross-open', 30, '30', 'square-x', 130, '130',
            'square-x-open', 31, '31', 'diamond-cross', 131, '131',
            'diamond-cross-open', 32, '32', 'diamond-x', 132, '132',
            'diamond-x-open', 33, '33', 'cross-thin', 133, '133',
            'cross-thin-open', 34, '34', 'x-thin', 134, '134',
            'x-thin-open', 35, '35', 'asterisk', 135, '135',
            'asterisk-open', 36, '36', 'hash', 136, '136',
            'hash-open', 236, '236', 'hash-dot', 336, '336',
            'hash-open-dot', 37, '37', 'y-up', 137, '137',
            'y-up-open', 38, '38', 'y-down', 138, '138',
            'y-down-open', 39, '39', 'y-left', 139, '139',
            'y-left-open', 40, '40', 'y-right', 140, '140',
            'y-right-open', 41, '41', 'line-ew', 141, '141',
            'line-ew-open', 42, '42', 'line-ns', 142, '142',
            'line-ns-open', 43, '43', 'line-ne', 143, '143',
            'line-ne-open', 44, '44', 'line-nw', 144, '144',
            'line-nw-open', 45, '45', 'arrow-up', 145, '145',
            'arrow-up-open', 46, '46', 'arrow-down', 146, '146',
            'arrow-down-open', 47, '47', 'arrow-left', 147, '147',
            'arrow-left-open', 48, '48', 'arrow-right', 148, '148',
            'arrow-right-open', 49, '49', 'arrow-bar-up', 149, '149',
            'arrow-bar-up-open', 50, '50', 'arrow-bar-down', 150,
            '150', 'arrow-bar-down-open', 51, '51', 'arrow-bar-left',
            151, '151', 'arrow-bar-left-open', 52, '52',
            'arrow-bar-right', 152, '152', 'arrow-bar-right-open', 53,
            '53', 'arrow', 153, '153', 'arrow-open', 54, '54',
            'arrow-wide', 154, '154', 'arrow-wide-open']

            '''

'\n            the followig maybe used when creating new plots\n            ===============================================\n            spin_dependency = row_data[\'spin_dependency\']\n            result_type = row_data[\'result_type\']\n            marker_size = 4\n            marker_color = row[\'trace_color_default\']\n            if spin_dependency == \'SI\':\n                trace_mode_value = \'lines\'\n                line_color = row[\'trace_color_default\']\n                line_style = dict(color=line_color)\n            elif spin_dependency == \'SD\':\n                trace_mode_value = \'lines\'\n                line_width = 4\n                line_format = \'dash\'\n                line_color = row[\'trace_color_default\']\n                line_style = dict(width= line_width ,dash= line_format ,color = line_color )\n            else:\n                mode_value = \'lines\'\n                line_style = None\n\n            if result_type == \'Th\':\n                fill_st

## Create Actual Chart

In [32]:
def create_actual_plot(plot_name_in, plot_id_in, old_plot_id_in, data_display_df_in, data_data_df_in):
    ## requirement to keep plots square
    #if screen_height_in > screen_width_in:
    #    plot_square_dimensions =
    #plot_square_dimensions = self.screen_height / 2
    plot_square_dimensions = 600

    m1 = go.layout.Margin(l=20,r=10,b=20,t=20,pad=0)
    hw = go.Layout(autosize=False,width=plot_square_dimensions,height=plot_square_dimensions)
    y_title_text = r"$\text{Cross Section [cm}^{2}\text{] (normalized to nucleon)}$"

    y1 = go.layout.YAxis(title=y_title_text,
                        title_standoff = 0,
                        #range=[start_y_range,stop_y_range],
                        type="log",
                        titlefont=go.layout.yaxis.title.Font(color='SteelBlue'))

    x_title_text = r"$\text{WIMP Mass [GeV}/c^{2}]$"
    x1 = go.layout.XAxis(title=x_title_text,
                        title_standoff = 0,
                        type="log",
                        #type="linear",
                        #range=[start_x_range,stop_x_range],
                        titlefont=go.layout.xaxis.title.Font(color='SteelBlue'))


    ##title1=go.layout.Title(text="Dark Matter Detection Results")

    fig_chart_populated = go.Figure(
        data=[go.Scatter(x=[], y=[])],
        layout=go.Layout(
            margin=m1,
            yaxis= y1,
            xaxis= x1
        )
    )

    plot_title = plot_name_in + " - P: " + str(plot_id_in) + " - O: " + str(old_plot_id_in)

    fig_chart_populated.update_layout(
        title=dict(text=plot_title ,font=dict(size=16),automargin=True,yref='paper')
    )

    fig_chart_populated.update_layout(hw)

    for index, row in data_display_df_in.iterrows():
        #print(row['limit_id'])
        data_id_selected = row['data_id']
        #print('selected data_id >>', data_id_selected)
        #data_about_selected_df = self.data_about_df[self.data_about_df['data_id']==data_id_selected].copy()
        data_display_selected_df_in = data_display_df_in[data_display_df_in['data_id']==data_id_selected].copy()
        data_data_selected_df_in = data_data_df_in[data_data_df_in['data_id']==data_id_selected].copy()
        # df[(df.year == 2013) & (df.country.isin(['US', 'FR', 'ES']))]
        for index, row in data_display_selected_df_in.iterrows():

            #trace_features : name, spin_dependency, result_type, mode_value, line_color, line_width, line_format, fill_style, fill_color, opacity_style, marker_size, marker_color
            trace_data = data_data_selected_df_in[(data_data_selected_df_in['data_id']==row['data_id']) \
                                            & (data_data_selected_df_in['trace_id']==row['trace_id'])].copy()
            #print(trace_data)
            #print("trace_data >>>>" , trace_data)
            row_data = row.copy()
            trace_name = row_data['trace_name']
            #print(data_id_selected, trace_name )
            mode = row['mode_value']
            if row['line_format'] == 'none':
                line_style = 'solid'
            else:
                line_style = row['line_format']
            line_width = row['line_width']
            line_color = row['line_color']
            opacity = row['opacity_style']
            fill_color = row['fill_color']
            symbol = 'circle'
            if row['symbol'] == 'triangle':
                symbol = 'triangle-up'
                mode = 'lines+markers'
            elif row['symbol'] == 'pent':
                symbol = 'pentagon'
                mode = 'lines+markers'
            elif row['symbol'] == 'dot':
                symbol = 'circle'
                mode = 'lines+markers'
            elif row['symbol'] == 'none':
                mode = 'lines'
            else:
                symbol = row['symbol']
                mode = 'lines+markers'
            marker_size = row['marker_size']
            marker_color = row['marker_color']
            fill_style = row['fill_style']
            if fill_style != 'fill':
                fig_chart_populated.add_trace(go.Scatter(x=trace_data['masses'],
                                              y=trace_data['cross_sections'],
                                              name=trace_name,
                                              #line={'dash':line_style},
                                              mode=mode,
                                              #mode='lines', # 'lines' or 'markers'
                                              #mode=trace_mode_value,
                                              line=dict(width=line_width,dash=line_style,color=line_color),
                                              #line=dict(width=1,dash=None,color=row['line_color']),
                                              #fill='toself',
                                              #fill=None,
                                              #opacity=opacity,
                                              #fillcolor='blue',
                                              #fillcolor = fill_color,
                                              #marker_symbol=None,
                                              #marker = None,
                                              marker_symbol=symbol,
                                              #marker_symbol='square',
                                              marker=dict(size=marker_size,color=marker_color),
                                              #legendgroup=str(row['limit_id']),  # this can be any string, not just "group"
                                              #legendgrouptitle_text=str(row['limit_id']),
                                              #########
                                              hovertemplate =
                                                  '<i><b>Cross Section</b></i>: %{y:.4E}'+
                                                  '<br><b>Masses</b>: %{x}<br>',
                                              #########
                                              text=trace_name,
                                              showlegend=False,
                                                  ))
            else:
                ## filled trace

                ##
                '''
                mode=mode,
                                              #opacity=opacity,
                                              opacity=0.5,
                                              #fillcolor = f"'{fill_color}'",
                                              fillcolor = fill_color,
                                              fill='toself',
                                              #text=row[current_column],
                                              line=dict(width=4,dash=line_format,color=row['line_color']),
                '''
                ##
                opacity_in = 0.01
                #fill_string = f"\'rgba(%u,%u,%u,{opacity_in})\'" % (100, 200, 255)
                fill_string = f"rgba(%u,%u,%u,{opacity_in})" % (255, 0, 0)
                print('fill_string>>>>>>>>>>>>>', fill_string)

                fig_chart_populated.add_trace(go.Scatter(x=trace_data['masses'],
                                              y=trace_data['cross_sections'],
                                              name=trace_name,
                                              mode='lines', # 'lines' or 'markers'
                                              line=dict(width=line_width,color=fill_color),
                                              fill='toself',
                                              #opacity=opacity,
                                              #opacity=0.5,
                                              #fillcolor = "'" & fill_color & "'",
                                              #fillcolor='rgba(255, 0, 0, .1)',
                                              #fillcolor  = fill_string,
                                              fillcolor = fill_color,
                                              ######### '%{x},%{y}<br>' +
                                              hovertemplate =
                                                  '<i>Cross Section</i>: %{y:.4E}'+
                                                  '<br><b>Masses</b>: %{x}<br>',
                                              #########
                                              text=trace_name,
                                              showlegend=False,
                                                  ))
    return fig_chart_populated
fig_chart_1 = create_actual_plot(plot_name, plot_id, old_plot_id, data_display_df, data_data_df )
fig_chart_1.show()

In [33]:
#data_data_df['data_id'].unique()
#data_data_df[data_data_df['data_id']==191]

In [34]:
#min_x = fig_chart_1['layout']['xaxis']['range'][0]
#max_x = fig_chart_1['layout']['xaxis']['range'][1]
#data_data_selected_df = data_data_df[data_data_df['data_id']==191].copy()
#data_data_selected_df

In [35]:
## testing main code
##if result_type == 'Th' then fill else nofil
##if spin_dependency = 'SD' then dash
##if spin_dependency = 'SI' then line



# The Allowed Symbols

# Create Legend

In [36]:
def create_char_legend(data_display_df_in):
    rows_list = list(range(1,20))
    cols_list = list(range(1,6))

    table_rows=20
    table_cols=5

    fig_legend_out = make_subplots(
                      column_titles = ['data_id','trace_id','trace_name','line_format','fill'],
                      rows=table_rows,
                      cols=table_cols,
                      horizontal_spacing = 0.00,
                      vertical_spacing = 0.00,
                      #subplot_titles=(titles)
                      column_widths=[0.1,0.1,0.6,0.1,0.1],)
    fig_legend_out.update_layout(
        #    autosize=False,
        #    width=800,
        #    height=200,
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=20,
                pad=0
            ),
            paper_bgcolor="LightSteelBlue",
        )

    fig_legend_out.update_xaxes(showgrid=False)
    fig_legend_out.update_yaxes(showgrid=False)
    #legend
    fig_legend_out.update_layout(showlegend=False)
    #x axis
    fig_legend_out.update_xaxes(visible=False)
    #y axis
    fig_legend_out.update_yaxes(visible=False)

    fig_legend_out.data = []
    #fig_legend_out.show()

    # Any changes to the fig must be applied to the DataFrame as the dataframe
    # will be used when the plot is saved.

    rowloop = 0
    for index, row in data_display_df_in.iterrows():
        mode = row['mode_value']
        if row['line_format'] == 'none':
            line_style = 'solid'
        else:
            line_style = row['line_format']
        line_width = row['line_width']
        line_color = row['line_color']
        opacity = row['opacity_style']
        fill_color = row['fill_color']
        if row['symbol'] == 'triangle':
            symbol = 'triangle-up'
        elif row['symbol'] == 'pent':
            symbol = 'pentagon'
        elif row['symbol'] == 'none':
            symbol = 'circle'
        else:
            symbol = row['symbol']
        marker_size = row['marker_size']
        marker_color = row['marker_color']
        fill_style = row['fill_style']
        #print(row['c1'], row['c2'])
        rowloop +=1
        for c in cols_list: #enumerate here to get access to i
            # STEP 2, notice position of arguments!
            table_column_names = ['data_id','trace_id','trace_name','line_format','fill']
            scatter_mode_list = ['text-number','text-number','text-text','lines','fill']
            #current_column = table_column_names[c-1]
            #current_mode = scatter_mode_list[c-1]
            current_column = table_column_names[c-1]
            current_mode = scatter_mode_list[c-1]
            if current_mode =='lines':
                if row['line_format'] == 'none':
                    line_format = 'solid'
                elif row['symbol'] != 'none':
                    mode = 'lines+markers'
                else:
                    line_format = row['line_format']
                    mode = 'lines'
                fig_legend_out.add_trace(go.Scatter(x=[1,2,3],
                                          y=[1,1,1],
                                          mode=mode,
                                          marker_symbol=symbol,
                                          marker=dict(
                                                  size=10,
                                                  color=row['symbol_color'],#set color equal to a variable
                                                  #colorscale='Viridis', # one of plotly colorscales
                                                  showscale=False,
                                                  ),
                                          #text=row[current_column],
                                          line=dict(width=4,dash=line_format,color=row['line_color']),
                                        ),
                              row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                              col=c)
            if current_mode =='text-text':
                fig_legend_out.add_trace(go.Scatter(x=[1,2],
                                          textposition='middle right',
                                          y=[1,1],
                                          mode='text',
                                          text=[row[current_column],'']
                                        ),
                              row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                              col=c)

            if current_mode =='fill':
                mode = 'lines'
                if fill_style != 'none':
                    fig_legend_out.add_trace(go.Scatter(x=[0,1,1,0,0],
                                              y=[0,0,1,1,0],
                                              mode=mode,
                                              opacity=opacity,
                                              fillcolor = fill_color,
                                              fill='toself',
                                              #text=row[current_column],
                                              line=dict(width=4,dash=line_format,color=row['line_color']),
                                            ),
                                  row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                                  col=c)
                else:
                    pass

            if current_mode =='text-number':
                fig_legend_out.add_trace(go.Scatter(x=[1],
                                          textposition='middle right',
                                          y=[1],
                                          mode='text',
                                          text=row[current_column]
                                        ),
                              row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                              col=c)
            if current_mode =='markers':
                fig_legend_out.add_trace(go.Scatter(x=[1],
                                        y=[1],
                                        mode=current_mode,
                                        #text=row[current_column],
                                        marker_symbol=symbol,
                                        marker=dict(
                                                  size=10,
                                                  color=row['symbol_color'],#set color equal to a variable
                                                  #colorscale='Viridis', # one of plotly colorscales
                                                  showscale=False,
                                                  )
                                        ),
                              row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                              col=c)

            fig_legend_out.update_xaxes(showgrid=False)
            fig_legend_out.update_yaxes(showgrid=False)
            #legend
            fig_legend_out.update_layout(showlegend=False)
            #x axis
            fig_legend_out.update_xaxes(visible=False)
            #y axis
            fig_legend_out.update_yaxes(visible=False)

    return fig_legend_out

legend_fig = create_char_legend(data_display_df)
legend_fig.show()

In [37]:
data_display_df.columns

Index(['id', 'subject', 'name', 'limit_id', 'data_id', 'old_plot_id',
       'plot_id', 'series_id', 'data_label', 'x_units', 'y_units', 'x_rescale',
       'y_rescale', 'trace_id', 'trace_name', 'line_color', 'symbol_color',
       'fill_color', 'line', 'symbol', 'spin_dependency', 'result_type',
       'mode_value', 'line_width', 'line_format', 'fill_style',
       'opacity_style', 'marker_size', 'marker_color', 'updated_at', 'row_id'],
      dtype='object')

## Legend Table Dimensions

# Create the Sub Plots for Each Legend Table Cell

In [38]:
'''
fig_legend_out.update_layout(yaxis=dict(range=[0,1]))
fig_legend_out.update_layout(xaxis=dict(range=[0,1]))
#fig_legend_out.update_xaxes(matches=None, showticklabels=True)
#fig_legend_out.update_yaxes(matches=None, showticklabels=True)
rowloop = 0
#fig_legend_out.update_layout(xaxis_range=[1,2])

for index, row in data_display_df.iterrows():
    rowloop +=1
    for c in cols_list: #enumerate here to get access to i
            # STEP 2, notice position of arguments!
            table_column_names = ['data_id','trace_id','trace_name','line_format','symbol']
            scatter_mode_list = ['text-number','text-number','text-text','lines','lines']
            #current_column = table_column_names[c-1]
            #current_mode = scatter_mode_list[c-1]
            current_column = table_column_names[c-1]
            current_mode = scatter_mode_list[c-1]
            opacity = 1
            fig_legend_out.add_trace(go.Scatter(x=[0,1,1,0,0],
                                    y=[0,0,1,1,0],
                                    mode='lines',
                                    opacity=opacity,
                                    #fillcolor='blue',
                                    fillcolor = row['line_color'],
                                    fill='toself',
                                    #text=row[current_column],
                                    line=dict(width=4,dash=line_format,color=row['line_color']),
                                  ),
                        row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                        col=c)
'''

"\nfig_legend_out.update_layout(yaxis=dict(range=[0,1]))\nfig_legend_out.update_layout(xaxis=dict(range=[0,1]))\n#fig_legend_out.update_xaxes(matches=None, showticklabels=True)\n#fig_legend_out.update_yaxes(matches=None, showticklabels=True)\nrowloop = 0\n#fig_legend_out.update_layout(xaxis_range=[1,2])\n\nfor index, row in data_display_df.iterrows():\n    rowloop +=1\n    for c in cols_list: #enumerate here to get access to i\n            # STEP 2, notice position of arguments!\n            table_column_names = ['data_id','trace_id','trace_name','line_format','symbol']\n            scatter_mode_list = ['text-number','text-number','text-text','lines','lines']\n            #current_column = table_column_names[c-1]\n            #current_mode = scatter_mode_list[c-1]\n            current_column = table_column_names[c-1]\n            current_mode = scatter_mode_list[c-1]\n            opacity = 1\n            fig_legend_out.add_trace(go.Scatter(x=[0,1,1,0,0],\n                              

In [39]:
#cols_list
len(data_display_df)

4

# Populate the Legend

In [40]:
'''rowloop = 0
cols_list =
for index, row in data_display_df.iterrows():
    #print(row['c1'], row['c2'])
    rowloop +=1
    for c in cols_list: #enumerate here to get access to i
        # STEP 2, notice position of arguments!
        table_column_names = ['data_id','trace_id','trace_name','line_format','symbol'],
        scatter_mode_list = ['text-number','text-number','text-text','lines','markers']
        current_column = table_column_names[c-1]
        current_mode = scatter_mode_list[c-1]
        if current_mode =='lines':
            fig_legend_out.add_trace(go.Scatter(x=[1,2],
                                      y=[1,1],
                                      mode=current_mode,
                                      #text=row[current_column],
                                      line=dict(width=4,dash=row['line_format'],color=row['line_color']),
                                    ),
                          row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                          col=c)
        if current_mode =='text-text':
            fig_legend_out.add_trace(go.Scatter(x=[1,2],
                                      textposition='middle right',
                                      y=[1,1],
                                      mode='text',
                                      text=[row[current_column],'']
                                    ),
                          row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                          col=c)

        if current_mode =='text-number':
            fig_legend_out.add_trace(go.Scatter(x=[1],
                                      textposition='middle right',
                                      y=[1],
                                      mode='text',
                                      text=row[current_column]
                                    ),
                          row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                          col=c)
        if current_mode =='markers':
            fig_legend_out.add_trace(go.Scatter(x=[1],
                                    y=[1],
                                    mode=current_mode,
                                    #text=row[current_column],
                                    marker_symbol=row['symbol'],
                                    marker=dict(
                                    size=10,
                                    color=row['symbol_color'],#set color equal to a variable
                                    colorscale='Viridis', # one of plotly colorscales
                                    showscale=False,
                                    )
                                    ),
                          row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                          col=c)

        fig_legend_out.update_xaxes(showgrid=False)
        fig_legend_out.update_yaxes(showgrid=False)
        #legend
        fig_legend_out.update_layout(showlegend=False)
        #x axis
        fig_legend_out.update_xaxes(visible=False)
        #y axis
        fig_legend_out.update_yaxes(visible=False)

fig_legend_out.show()'''

"rowloop = 0\ncols_list =\nfor index, row in data_display_df.iterrows():\n    #print(row['c1'], row['c2'])\n    rowloop +=1\n    for c in cols_list: #enumerate here to get access to i\n        # STEP 2, notice position of arguments!\n        table_column_names = ['data_id','trace_id','trace_name','line_format','symbol'],\n        scatter_mode_list = ['text-number','text-number','text-text','lines','markers']\n        current_column = table_column_names[c-1]\n        current_mode = scatter_mode_list[c-1]\n        if current_mode =='lines':\n            fig_legend_out.add_trace(go.Scatter(x=[1,2],\n                                      y=[1,1],\n                                      mode=current_mode,\n                                      #text=row[current_column],\n                                      line=dict(width=4,dash=row['line_format'],color=row['line_color']),\n                                    ),\n                          row=rowloop, #index for the subplot, i+1 because pl

# Display the Legend

In [41]:
#fig_legend_out.show()